# Análisis de campañas maliciosas

Lo interesante de los grafos es que nos permiten visualizar mucha información y nos pueden dar un punto de partida sobre como abordar nuestros análisis, que puede ser desde encontrar semejanzas con otras campañas vistas anteriormente o quizá identificar si se trata de una campaña que no tiene relación con algo visto anteriormente

Así que para esta caso volveremos a utilizar la librería `networkx`, para generar los grafos, además de algunas otras librerías que nos van a permitir manipular archivos.
Particularmente, en esta parte del workshop propongo un análisis a partir de los iconos de las muestras de malware. Esta pequeña pieza de información nos puede aportar bastante información, sobre todo cuando se trata de campañas que utilizan la ingeniería social como vector de propagación, por ejemplo ocultando el código malicioso en archivos con doble extensión.

In [6]:
#!/usr/bin/python

import sys
import os
import logging
import networkx
import tempfile
from subprocess import check_output
from networkx.drawing.nx_agraph import write_dot
from networkx.algorithms import bipartite

network = networkx.Graph()

Para este caso voy a utilizar una serie de muestras detectadas por las tecnologías de ESET durante la primera semana de septiembre en algunos países de Latinoamérica. Si bien por temas legales no puedo compartir las muestras que utilicé en el análisis, al final les voy a dejar los *hashes* de las muestras utilizadas y también voy a compartir algunos de los resultados para entender de que manera se pueden usar estos análisis.

La idea es extraer información de los iconos de las diferentes muestras para analizar, y para esto nos valemos de herramientas como `wrestool` e `icotool` para recolectarla y almacenarla para su análisis

In [2]:
ruta='/Agrega la ruta donde están las muestras'
objetos=[]
carpeta=os.listdir(ruta)

for muestra in carpeta:
    
    carpetas_tmp=tempfile.mkdtemp()
    
    icondir = os.path.join(carpetas_tmp,"icons")
    bitmapdir = os.path.join(carpetas_tmp,"bitmaps")
    raw_resources = os.path.join(carpetas_tmp,"raw")
    
    for directory in [icondir,bitmapdir,raw_resources]:
        os.mkdir(directory)
            
    target_binary=os.path.join(ruta,muestra)

    rawcmd = "wrestool -x {0} -o {1}".format(target_binary,raw_resources)
    bmpcmd = "mv {0}/*.bmp {1}".format(raw_resources,bitmapdir)
    icocmd = "icotool -x {0}/*.ico -o {1}".format(raw_resources,icondir)
       
    for dirname in [icondir,bitmapdir]:
        for pathimages in os.listdir(dirname):
            logging.info(pathimages)
            pathashes = os.path.join(dirname,pathimages)
                
            comando = "sha1sum {0}".format(pathashes)
            imagehash = check_output(comando, shell=True).decode("utf-8")[0:40]
            if pathimages.endswith(".png"):
                objetos.append((pathashes,imagehash))
            if pathimages.endswith(".bmp"):
                objetos.append((pathashes,imagehash))
    
    for ruta_imagen, hash_imagen in objetos:
            
            if not hash_imagen in network:
                network.add_node(hash_imagen,image=ruta_imagen,label='',type='image',color='blue')
            node_name = (ruta_imagen.split("/")[-1]).split("_")[0]
            network.add_node(node_name,type="malware",color='red')
            network.add_edge(node_name,hash_imagen)    

##### No olviden monitorear la carpeta `/tmp`, después de varias rondas de análisis puede ocupar mucho espacio en disco

Ahora con toda la información procesada y separada podemos escribir los archivos *.dot* para luego graficar. Como había mencionado anteriormente en este caso utilizo *fdp* de `GraphViz`, hay otras opciones como *sfdp* o *neato* que nos pueden dar otras opciones para distribuir los grafos. 
Si bien `pandas` ofrece también muy buenas alternativas de visualización, a medida que crece la cantidad de nodos y aristas del grafo herramientas como `GraphViz` dan mejores resultados.

In [3]:
grafo_iconos='/Agrega la ruta donde vas a dejar las archivos/grafoICO.dot'
grafo_bipartito='/Agrega la ruta donde vas a dejar las archivos/grafo_bipartito.dot'

write_dot(network, grafo_iconos)

malware = set(n for n,d in network.nodes(data=True) if d['type']=='malware')

malware_network = bipartite.projected_graph(network, malware)
write_dot(malware_network,grafo_bipartito)

Utilizando la herramienta *fdp* de `GraphViz` se puede obtener la representación de nuestro grafo

`$fdp grafoICO.dot -T png -o grafoICO.png -Goverlap=false`

<a href="https://ibb.co/5MLMQTB"><img src="https://i.ibb.co/kHXHLKQ/grafo4-mex.png" alt="grafo4-mex" border="0"></a>

Otro grafo

<a href="https://ibb.co/NnXkNx6"><img src="https://i.ibb.co/YT6Kd27/grafo-iconos-malware-col.png" alt="grafo-iconos-malware-col" border="0"></a>

Otro grafo más

<a href="https://ibb.co/tBRk9LQ"><img src="https://i.ibb.co/H7Z1jhB/grafo-iconos-malware-mex.png" alt="grafo-iconos-malware-mex" border="0"></a>



## Grafos bipartitos


`$fdp grafo_bipartito.dot -T png -o grafo_bipartito.png -Goverlap=false`

### Grafo de detecciones en MX
<a href="https://ibb.co/wrzfpCT"><img src="https://i.ibb.co/qFrvm9z/bipartite1.jpg" alt="bipartite1" border="0"></a>

#### Un poco más detalle
<a href="https://ibb.co/bKfQk3J"><img src="https://i.ibb.co/d5hjRcg/bipartite-MX.jpg" alt="bipartite-MX" border="0"></a>

### Grafo de detecciones en CO
<a href="https://ibb.co/Cm1pm48"><img src="https://i.ibb.co/TYvDYJ2/bipartite2.jpg" alt="bipartite2" border="0"></a>

#### Un poco más detalle
<a href="https://ibb.co/t2dWmDn"><img src="https://i.ibb.co/DMqSVQm/bipartite-CO.jpg" alt="bipartite-CO" border="0"></a>